In [1]:
from app.service.text_service import TextAnalysisService
from app.service.image_service import ImageAnalysisService
from app.service.file_service import FileAnalysisService
from app.service.url_services import UrlAnalysisService
from app.service.label_service import LabelApplicationService, LabelManagementService

from bson import ObjectId

text_analysis_service = TextAnalysisService()
url_analysis_service = UrlAnalysisService()
file_analysis_service = FileAnalysisService()
image_analysis_service = ImageAnalysisService()

await text_analysis_service.process_text()
await url_analysis_service.process_urls()
await file_analysis_service.process_files()
await image_analysis_service.process_images()

2025-04-12 18:35:14,903 - ChartMind - INFO - Found 0 unprocessed texts.
2025-04-12 18:35:14,904 - ChartMind - INFO - Found no unprocessed texts.


KeyError: 'user_id'

In [1]:
from pymongo import MongoClient
from app.infrastructure.external.cloudflare_ai_service import CloudflareAIService
from app.infrastructure.daos.text_daos import TextDAO
from app.infrastructure.daos.image_daos import ImageDAO

In [9]:
llm_service = CloudflareAIService()

dao = TextDAO()
dao = ImageDAO()

query_text = "GPU"

result = await dao.full_text_search(query_text)

# query_vector = await llm_service.get_embedding(query_text)
# result = await dao.vector_search(query_vector)

In [ ]:
result